In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Étape 1 : Définir des modèles Pydantic pour des sorties structurées

Ces modèles définissent le **schéma** que les agents retourneront. Utiliser `response_format` avec Pydantic garantit :
- ✅ Extraction de données sécurisée par type
- ✅ Validation automatique
- ✅ Pas d'erreurs d'analyse provenant des réponses en texte libre
- ✅ Routage conditionnel simplifié basé sur les champs


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Étape 2 : Créer l'outil de réservation d'hôtel

Cet outil sera utilisé par l'**availability_agent** pour vérifier la disponibilité des chambres. Nous utilisons le décorateur `@ai_function` pour :
- Transformer une fonction Python en un outil accessible par l'IA
- Générer automatiquement un schéma JSON pour le LLM
- Gérer la validation des paramètres
- Permettre l'invocation automatique par les agents

Pour cette démonstration :
- **Stockholm, Seattle, Tokyo, Londres, Amsterdam** → Chambres disponibles ✅
- **Toutes les autres villes** → Pas de chambres ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Étape 3 : Définir les fonctions de condition pour le routage

Ces fonctions examinent la réponse de l'agent et déterminent quel chemin suivre dans le flux de travail.

**Modèle clé :**
1. Vérifiez si le message est `AgentExecutorResponse`
2. Analysez la sortie structurée (modèle Pydantic)
3. Retournez `True` ou `False` pour contrôler le routage

Le flux de travail évaluera ces conditions sur **les arêtes** pour décider quel exécuteur invoquer ensuite.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Étape 4 : Créer un exécuteur d'affichage personnalisé

Les exécuteurs sont des composants de workflow qui effectuent des transformations ou des effets secondaires. Nous utilisons le décorateur `@executor` pour créer un exécuteur personnalisé qui affiche le résultat final.

**Concepts clés :**
- `@executor(id="...")` - Enregistre une fonction en tant qu'exécuteur de workflow
- `WorkflowContext[Never, str]` - Indications de type pour les entrées/sorties
- `ctx.yield_output(...)` - Produit le résultat final du workflow


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Étape 5 : Charger les variables d'environnement

Configurez le client LLM. Cet exemple fonctionne avec :
- **Modèles GitHub** (Niveau gratuit avec un token GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Étape 6 : Créer des agents IA avec des sorties structurées

Nous créons **trois agents spécialisés**, chacun encapsulé dans un `AgentExecutor` :

1. **availability_agent** - Vérifie la disponibilité des hôtels à l'aide de l'outil
2. **alternative_agent** - Propose des villes alternatives (en cas d'absence de chambres)
3. **booking_agent** - Encourage la réservation (lorsque des chambres sont disponibles)

**Caractéristiques principales :**
- `tools=[hotel_booking]` - Fournit l'outil à l'agent
- `response_format=PydanticModel` - Impose une sortie JSON structurée
- `AgentExecutor(..., id="...")` - Encapsule l'agent pour une utilisation dans le flux de travail


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Étape 7 : Construire le flux de travail avec des transitions conditionnelles

Nous utilisons maintenant `WorkflowBuilder` pour construire le graphe avec un routage conditionnel :

**Structure du flux de travail :**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Méthodes clés :**
- `.set_start_executor(...)` - Définit le point d'entrée
- `.add_edge(from, to, condition=...)` - Ajoute une transition conditionnelle
- `.build()` - Finalise le flux de travail


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Étape 8 : Exécuter le cas de test 1 - Ville SANS disponibilité (Paris)

Testons le chemin **sans disponibilité** en demandant des hôtels à Paris (qui n'a aucune chambre dans notre simulation).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Étape 9 : Exécuter le cas de test 2 - Ville AVEC disponibilité (Stockholm)

Testons maintenant le chemin de **disponibilité** en demandant des hôtels à Stockholm (qui dispose de chambres dans notre simulation).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Points Clés et Prochaines Étapes

### ✅ Ce que vous avez appris :

1. **Modèle WorkflowBuilder**
   - Utilisez `.set_start_executor()` pour définir le point d'entrée
   - Utilisez `.add_edge(from, to, condition=...)` pour le routage conditionnel
   - Appelez `.build()` pour finaliser le workflow

2. **Routage Conditionnel**
   - Les fonctions de condition inspectent `AgentExecutorResponse`
   - Analysez les sorties structurées pour prendre des décisions de routage
   - Retournez `True` pour activer une connexion, `False` pour l'ignorer

3. **Intégration des Outils**
   - Utilisez `@ai_function` pour convertir des fonctions Python en outils d'IA
   - Les agents appellent automatiquement les outils lorsque nécessaire
   - Les outils retournent du JSON que les agents peuvent analyser

4. **Sorties Structurées**
   - Utilisez les modèles Pydantic pour une extraction de données sécurisée par type
   - Définissez `response_format=MyModel` lors de la création des agents
   - Analysez les réponses avec `Model.model_validate_json()`

5. **Exécuteurs Personnalisés**
   - Utilisez `@executor(id="...")` pour créer des composants de workflow
   - Les exécuteurs peuvent transformer des données ou effectuer des effets secondaires
   - Utilisez `ctx.yield_output()` pour produire des résultats de workflow

### 🚀 Applications Réelles :

- **Réservation de Voyages** : Vérifiez la disponibilité, proposez des alternatives, comparez les options
- **Service Client** : Orientez en fonction du type de problème, du sentiment, de la priorité
- **E-commerce** : Vérifiez les stocks, proposez des alternatives, traitez les commandes
- **Modération de Contenu** : Orientez en fonction des scores de toxicité, des signalements des utilisateurs
- **Workflows d'Approbation** : Orientez en fonction du montant, du rôle utilisateur, du niveau de risque
- **Traitement Multi-Étapes** : Orientez en fonction de la qualité des données, de leur complétude

### 📚 Prochaines Étapes :

- Ajoutez des conditions plus complexes (critères multiples)
- Implémentez des boucles avec la gestion de l'état du workflow
- Ajoutez des sous-workflows pour des composants réutilisables
- Intégrez des API réelles (réservation d'hôtel, systèmes d'inventaire)
- Ajoutez la gestion des erreurs et des chemins de secours
- Visualisez les workflows avec les outils de visualisation intégrés



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
